## Three Stock Monte

This exercise tests your ability to retrieve stock price data using the Alpaca API and simulate multiple portfolio compositions using Monte Carlo Simulation.

### Import Dependencies

In [1]:
# Import libraries and dependencies
import os
import pandas as pd
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

In [2]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv("api.env")

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

### Get Past ~4 Year's Worth of Stock Price Data via Alpaca API Call

In [4]:
# Set timeframe to "1Day"
timeframe = "1Day"

# Set start and end datetimes between now and 4 years ago.
start_date = pd.Timestamp("2017-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-05-01", tz="America/New_York").isoformat()

# Set the ticker information
tickers = ["V","NKE","XOM"]

# Get 4 year's worth of historical price data
# HINT: Set "limit" to at least 10000 so all ticker rows are captured from get_bars()
df_ticker = api.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

# Display sample data
df_ticker.head()

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2017-05-01 04:00:00+00:00,54.99,55.45,54.98,30493,55.43,5361441,55.190028,NKE
2017-05-02 04:00:00+00:00,55.07,55.16,54.79,41590,55.12,7792518,54.974022,NKE
2017-05-03 04:00:00+00:00,54.53,55.30,54.40,48477,55.17,10746396,54.612038,NKE
2017-05-04 04:00:00+00:00,54.47,54.83,54.20,39262,54.68,7310669,54.501759,NKE
2017-05-05 04:00:00+00:00,53.95,54.50,53.56,57957,54.41,14261080,53.898559,NKE


> Note: We're going to simulate five years of growth below, and so we might want to pull at least five years of data in order to do that. But we pulled four because of Alpaca's rate limits--we can only pull 1,000 rows per ticker at a time (approximately four years of data, with 252 tradable days per year). If we pull data with repeated calls in a `for` loop, however, we can get quite a few more years than what we've got above.  

In [5]:
# Reorganize the DataFrame
# Separate ticker data
T = df_ticker[df_ticker["symbol"]=="V"].drop("symbol", axis=1)
NKE = df_ticker[df_ticker["symbol"]=="NKE"].drop("symbol", axis=1)
XOM = df_ticker[df_ticker["symbol"]=="XOM"].drop("symbol", axis=1)

# Concatenate the ticker DataFrames
df_ticker = pd.concat([T, NKE, XOM], axis=1, keys=["V","NKE","XOM"])

# Display sample data
df_ticker.head()

V                                               \
                           close    high     low trade_count   open    volume   
timestamp                                                                       
2017-05-01 04:00:00+00:00  91.26  91.670  91.140       44585  91.29  10604089   
2017-05-02 04:00:00+00:00  92.54  92.750  91.530       55002  91.62  13327724   
2017-05-03 04:00:00+00:00  92.44  92.545  91.975       36001  92.40   5361856   
2017-05-04 04:00:00+00:00  92.62  92.850  92.250       34603  92.70   5890401   
2017-05-05 04:00:00+00:00  92.09  92.980  91.760       42726  92.90   9943195   

                                        NKE                ...         \
                                vwap  close   high    low  ...   open   
timestamp                                                  ...          
2017-05-01 04:00:00+00:00  91.335788  54.99  55.45  54.98  ...  55.43   
2017-05-02 04:00:00+00:00  92.354141  55.07  55.16  54.79  ...  55.12   
2017-05-03 04:00:00+00:00  92.295444  54.53  55.30  54.40  ...  55.17   
2017-05-04 04:00:00+00:00  92.584404  54.47  54.83  54.20  ...  54.68   
2017-05-05 04:00:00+00:00  92.099626  53.95  54.50  53.56  ...  54.41   

                                                  XOM                  \
                             volume       vwap  close     high    low   
timestamp                                                               
2017-05-01 04:00:00+00:00   5361441  55.190028  82.06  82.2100  81.48   
2017-05-02 04:00:00+00:00   7792518  54.974022  82.05  82.5900  81.90   
2017-05-03 04:00:00+00:00  10746396  54.612038  82.70  83.0699  81.91   
2017-05-04 04:00:00+00:00   7310669  54.501759  81.64  82.3600  81.40   
2017-05-05 04:00:00+00:00  14261080  53.898559  82.02  82.2350  81.61   

                                                                   
                          trade_count   open    volume       vwap  
timestamp                                                          
2017-05-01 04:00:00+00:00       49225  81.51   9769374  81.945427  
2017-05-02 04:00:00+00:00       54236  82.14  10044837  82.185586  
2017-05-03 04:00:00+00:00       57309  81.95  11189766  82.631674  
2017-05-04 04:00:00+00:00       86290  82.29  17175853  81.728301  
2017-05-05 04:00:00+00:00       59390  81.64  11203370  81.940459  

[5 rows x 21 columns]

### Simulate five year portfolio growth with evenly-distributed stock investments

In [6]:
# Configure a Monte Carlo simulation to forecast five years cumulative returns
MC_even_dist = MCSimulation(
    portfolio_data = df_ticker,
    weights = [.33,.33,.33],
    num_simulation = 1000,
    num_trading_days = 252*5
)

# Print the simulation input data
MC_even_dist.portfolio_data.head()

T                                             \
                           close   high    low trade_count   open    volume   
timestamp                                                                     
2017-05-01 04:00:00+00:00  39.10  39.70  39.06       95236  39.68  27137598   
2017-05-02 04:00:00+00:00  38.95  39.22  38.88       86239  39.14  26346738   
2017-05-03 04:00:00+00:00  38.40  38.98  38.15      113921  38.94  36621125   
2017-05-04 04:00:00+00:00  38.01  38.41  37.63      118192  38.41  32681751   
2017-05-05 04:00:00+00:00  38.56  38.71  38.03       82036  38.08  23262074   

                                                     NKE         ...  \
                                vwap daily_return  close   high  ...   
timestamp                                                        ...   
2017-05-01 04:00:00+00:00  39.283136          NaN  54.99  55.45  ...   
2017-05-02 04:00:00+00:00  38.996146    -0.003836  55.07  55.16  ...   
2017-05-03 04:00:00+00:00  38.426498    -0.014121  54.53  55.30  ...   
2017-05-04 04:00:00+00:00  37.969998    -0.010156  54.47  54.83  ...   
2017-05-05 04:00:00+00:00  38.426797     0.014470  53.95  54.50  ...   

                                                     XOM                  \
                                vwap daily_return  close     high    low   
timestamp                                                                  
2017-05-01 04:00:00+00:00  55.190028          NaN  82.06  82.2100  81.48   
2017-05-02 04:00:00+00:00  54.974022     0.001455  82.05  82.5900  81.90   
2017-05-03 04:00:00+00:00  54.612038    -0.009806  82.70  83.0699  81.91   
2017-05-04 04:00:00+00:00  54.501759    -0.001100  81.64  82.3600  81.40   
2017-05-05 04:00:00+00:00  53.898559    -0.009547  82.02  82.2350  81.61   

                                                                                
                          trade_count   open    volume       vwap daily_return  
timestamp                                                                       
2017-05-01 04:00:00+00:00       49225  81.51   9769374  81.945427          NaN  
2017-05-02 04:00:00+00:00       54236  82.14  10044837  82.185586    -0.000122  
2017-05-03 04:00:00+00:00       57309  81.95  11189766  82.631674     0.007922  
2017-05-04 04:00:00+00:00       86290  82.29  17175853  81.728301    -0.012817  
2017-05-05 04:00:00+00:00       59390  81.64  11203370  81.940459     0.004655  

[5 rows x 24 columns]

In [7]:
# Run a Monte Carlo simulation to forecast five years cumulative returns
MC_even_dist.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.999192,0.998455,0.999522,1.006994,1.003011,0.997172,0.988902,1.000368,0.985623,1.020365,...,0.995469,0.992349,1.009512,1.002792,0.994119,1.005556,0.997079,1.010592,1.006384,0.995117
2,1.000967,0.991836,1.001784,1.008165,0.999072,0.990421,1.002763,1.013229,0.986938,1.013496,...,1.001163,1.004802,1.014912,1.002407,1.013698,0.987964,0.986316,0.998597,1.011956,0.989784
3,1.016021,0.963408,1.012257,1.034774,0.990679,0.985738,1.003699,1.010271,0.997518,1.014839,...,1.001116,1.015495,1.028140,1.000539,1.006914,0.978771,1.002921,0.986813,1.015700,0.990711
4,1.017229,0.975007,0.994411,1.035505,0.985183,0.992617,1.001458,1.004533,1.005899,1.018161,...,0.976872,1.020989,1.033851,0.982211,1.007443,0.978868,0.994821,0.988594,1.008957,1.000769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,1.649021,1.196718,0.957053,1.398495,1.243152,0.923367,1.156809,1.354757,1.656735,0.922346,...,0.860181,1.323879,1.050677,1.206132,1.389435,1.407146,2.004083,1.672072,0.891197,3.187410
1257,1.672322,1.190720,0.953010,1.431082,1.223814,0.925345,1.142601,1.355008,1.663498,0.915643,...,0.862258,1.327793,1.056134,1.207881,1.401418,1.410874,2.024079,1.674158,0.888572,3.224948
1258,1.698014,1.194924,0.954092,1.427317,1.233807,0.913842,1.129299,1.340770,1.645578,0.904080,...,0.853095,1.345343,1.065404,1.213718,1.411635,1.414441,2.009580,1.685916,0.876782,3.178337
1259,1.660662,1.216426,0.948094,1.438724,1.249390,0.920986,1.136284,1.345664,1.643548,0.902387,...,0.857600,1.376521,1.062669,1.193543,1.445170,1.389631,2.001540,1.682086,0.882251,3.150974


In [8]:
# Plot simulation outcomes
line_plot = MC_even_dist.plot_simulation()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot = MC_even_dist.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
even_tbl = MC_even_dist.summarize_cumulative_return()

# Print summary statistics
print(even_tbl)

In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $15,000 investments in stocks
even_ci_lower = round(even_tbl[8]*15000,2)
even_ci_upper = round(even_tbl[9]*15000,2)

# Print results
print(f"There is a 95% chance that an initial investment of $15,000 in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${even_ci_lower} and ${even_ci_upper}.")

### Simulate five year portfolio growth with 60% AT&T stock

In [ ]:
# Configure a Monte Carlo simulation to forecast five years cumulative returns with 60% AT&T stock
MC_att = MCSimulation(
    portfolio_data = df_ticker,
    weights = [.20,.60,.20],
    num_simulation = 1000,
    num_trading_days = 252*5)

# Print the simulation input data
MC_att.portfolio_data.head()

In [ ]:
# Run a Monte Carlo simulation to forecast five years cumulative returns with 60% AT&T stock
MC_att.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
att_line_plot = MC_att.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
att_dist_plot = MC_att.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
att_tbl = MC_att.summarize_cumulative_return()

# Print summary statistics
print(att_tbl)

In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $15,000 investments
att_ci_lower = round(att_tbl[8]*15000,2)
att_ci_upper = round(att_tbl[9]*15000,2)

# Print results
print(f"There is a 95% chance that an initial investment of $15,000 in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${att_ci_lower} and ${att_ci_upper}.")

### Simulate five year portfolio growth with 60% Nike stock

In [ ]:
# Configure a Monte Carlo simulation to forecast five years cumulative returns with 60% Nike stock
MC_nike = MCSimulation(
    portfolio_data = df_ticker,
    weights = [.60,.20,.20],
    num_simulation = 1000,
    num_trading_days = 252*5
)

# Printing the simulation input data
MC_nike.portfolio_data.head()

In [ ]:
# Run a Monte Carlo simulation to forecast five years cumulative returns with 60% Nike stock
MC_nike.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
nike_line_plot = MC_nike.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
nike_dist_plot = MC_nike.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
nike_tbl = MC_nike.summarize_cumulative_return()

# Print summary statistics
print(nike_tbl)

In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $15,000 investments
nike_ci_lower = round(nike_tbl[8]*15000,2)
nike_ci_upper = round(nike_tbl[9]*15000,2)

# Print results
print(f"There is a 95% chance that an initial investment of $15,000 in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${nike_ci_lower} and ${nike_ci_upper}.")

### Simulate five year portfolio growth with 60% Exxon stock

In [ ]:
# Configuring a Monte Carlo simulation to forecast five years cumulative returns
MC_exxon = MCSimulation(
    portfolio_data = df_ticker,
    weights = [.20,.20,.60],
    num_simulation = 1000,
    num_trading_days = 252*5)

# Printing the simulation input data
MC_exxon.portfolio_data

In [ ]:
# Run a Monte Carlo simulation to forecast five years cumulative returns with 60% Exxon stock
MC_exxon.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
exxon_line_plot = MC_exxon.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
exxon_dist_plot = MC_exxon.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
exxon_tbl = MC_exxon.summarize_cumulative_return()

# Print summary statistics
print(exxon_tbl)

In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $15,000 investments
exxon_ci_lower = round(exxon_tbl[8]*15000,2)
exxon_ci_upper = round(exxon_tbl[9]*15000,2)

# Print results
print(f"There is a 95% chance that an initial investment of $15,000 in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${exxon_ci_lower} and ${exxon_ci_upper}.")

### Summarize findings across all 4 simulations

In [ ]:
# Even weighted stocks
print("Even weighted stocks")
print(f"There is a 95% chance that an initial investment of $15,000 in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${even_ci_lower} and ${even_ci_upper}.")
print("*"*50)

# 60% for AT&T
print("60% for AT&T")
print(f"There is a 95% chance that an initial investment of $15,000 in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${att_ci_lower} and ${att_ci_upper}.")
print("*"*50)

# 60% for Nike
print("60% for Nike")
print(f"There is a 95% chance that an initial investment of $15,000 in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${nike_ci_lower} and ${nike_ci_upper}.")
print("*"*50)

# 60% for Exxon
print("60% for Exxon")
print(f"There is a 95% chance that an initial investment of $15,000 in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${exxon_ci_lower} and ${exxon_ci_upper}.")
print("*"*50)

Looking across all four simulations, the portfolio breakdown with the greatest chance of success looks to be the portfolio with a majority of AT&T stock. Although all four portfolios have a chance to lose money, the Nike portfolio is roughly the same level of risk with far more upside potential.